In [7]:
from ConfigSpace import ConfigurationSpace
from ConfigSpace.hyperparameters import CategoricalHyperparameter, UniformFloatHyperparameter, UniformIntegerHyperparameter
from ConfigSpace.conditions import InCondition
from ConfigSpace.forbidden import ForbiddenEqualsClause

def get_configspace():
    learning_rate = CategoricalHyperparameter("learning_rate", choices=["adaptive", "constant"])
    alpha = UniformFloatHyperparameter("alpha", lower=1e-7, upper=1e-1, log=True)
    max_iter = UniformIntegerHyperparameter("max_iter", lower=100, upper=1000)
    eta0 = UniformFloatHyperparameter("eta0", lower=1e-4, upper=1.0)
    early_stopping = CategoricalHyperparameter("early_stopping", choices=[True, False], default_value=True)

    config_space = ConfigurationSpace()
    config_space.add_hyperparameters([learning_rate, alpha, max_iter, eta0, early_stopping])

    eta0_condition = InCondition(eta0, learning_rate, values=["constant"])
    config_space.add_condition(eta0_condition)

    forbidden_clause = ForbiddenEqualsClause(eta0, None, learning_rate)
    config_space.add_forbidden_clause(forbidden_clause)

    return config_space


In [8]:
config = get_configspace()

/var/folders/27/qmbdy4fs531c8887h6nfgych0000gn/T/ipykernel_77942/4245066465.py:14: DeprecationWarning: Please use `space.add(hyperparameters)`
  config_space.add_hyperparameters([learning_rate, alpha, max_iter, eta0, early_stopping])
/var/folders/27/qmbdy4fs531c8887h6nfgych0000gn/T/ipykernel_77942/4245066465.py:17: DeprecationWarning: Please use `space.add(condition)`
  config_space.add_condition(eta0_condition)


TypeError: ForbiddenEqualsClause.__init__() takes 3 positional arguments but 4 were given